## 1. Создание предсказательной модели и оценивание её качества.

Считаем данные из скачанной открытой гугл-папки

In [1]:
import os
import pandas as pd
path = r'/Users/romanova/Downloads/mini-course-public'
data = {}

Data = []
Target = []

i=0
for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            #print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=';')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                #print(dir_entry_path.split("\\")[-1])
                pass

for dir_entry in sorted(os.listdir(path)):
    dir_entry_path = os.path.join(path, dir_entry)
    if os.path.isfile(dir_entry_path):
        i+=1
        with open(dir_entry_path, 'r') as my_file:
            #print(dir_entry_path.split("\\")[-1])
            try:
                df = pd.read_csv(my_file, delimiter=',')
                if df.shape[1] == 5:
                    Data.append(df)
                    Target.append(dir_entry_path.split("\\")[-1])
            except UnicodeDecodeError:
                #print(dir_entry_path.split("\\")[-1])
                pass

Переведём типы движений в числа

In [290]:
classes = {'тояни' : 0, 'месте' : 0, 'одьб' : 1, 'аг' : 1, 'ег' : 2, 'елоси' : 3, 'естн' : 4, 'одъ' : 4, 
           'втомо' : 5, 'ашин' : 5, 'метро' : 6, 'лектро' : 7, 'амок' : 8}

answers = {'стояние' : 0, 'ходьба' : 1, 'бег' : 2, 'велосипед' : 3, 'лестница' : 4, 
           'автомобиль' : 5, 'метро' : 6, 'электросамокат' : 7, 'самокат' : 8}

# стояние, ходьба, велосипед, лестница, бег, подъем, подъём, автомобиль, электросамокат, машина, метро

In [291]:
def get_action(name):
    for act in classes:
        if name.lower().find(act) != -1:
            return classes[act]
    return -1

Обработаем данные и извлечём признаки для создания модели

In [292]:
def rotation_matrix_from_vectors(vec1, vec2):
    a, b = (vec1 / numpy.linalg.norm(vec1)).reshape(3), (vec2 / numpy.linalg.norm(vec2)).reshape(3)
    v = numpy.cross(a, b)
    if any(v): #if not all zeros then 
        c = numpy.dot(a, b)
        s = numpy.linalg.norm(v)
        kmat = numpy.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
        return numpy.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

    else:
        return numpy.eye(3) #cross of all zeros only occurs on identical directions
    
def rotate(X, Y, Z, Mat):
    new_x = []
    new_y = []
    new_z = []
    
    for elem in zip(X,Y,Z):
        res = Mat.dot(elem)
        new_x.append(res[0])
        new_y.append(res[1])
        new_z.append(res[2])
        
    return new_x, new_y, new_z

def integrate(lst):
    res = []
    s = 0
    for elem in lst:
        s += elem
        res.append(s)
    return res

In [498]:
import pandas as pd
from matplotlib import pylab as plt
%pylab inline

def process_data(df, comma=True, cut=True, name = "", a=-1, b=-1):
    
# обрезаем начало и конец трека
    datFram_len = len(df) 
    toCut = round(datFram_len * 0.25)
    if a == -1:
        st = toCut
    else:
        st = a
            
    if b == -1:
        end = -toCut
    else:
        end = b
        
    if cut == False:
        st = 0
        end = -1
    
    flag = False
    if name.lower().find('ояни') != -1:
        print(name)
        flag = True
#     fig, ((ax1, ax2)) = plt.subplots(nrows=1, ncols=2, figsize=(15,6))

# приводим данные с Iphone к нужному формату
    if comma is True:
        df = df.applymap(lambda x: str(x).replace(',','.'))
        
        try:
            df['gFx'] = df['gFx'].astype(float)
            df['gFy'] = df['gFy'].astype(float)
            df['gFz'] = df['gFz'].astype(float)
        except KeyError:
            return [-1, -1, -1]
        
########################################################
# amplitude
########################################################

# вычисляем модуль (норму) ускорения и рисуем его
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5
#     ax2.plot(Acc)
#     plt.show()

# можно различить многие действия по амплитуде ускорения
    amplitude = Acc.max() - Acc.min()
#    print(' АМПЛИТУДА:', Acc.max()-Acc.min())
#    print()
    
            
########################################################
# преобразование Фурье
########################################################

    Fs = 250 #частота сбора данных
    Min = 0
    Acc = (df.gFx[st:end] ** 2 + df.gFy[st:end] ** 2 + df.gFz[st:end] ** 2) ** 0.5

    y = Acc.values
    if len(y) == 0:
        hesitation = -1
    else:
        n = len(y) # length of the signal
        k = np.arange(n)
        T = n/Fs
        frq = k/T # two sides frequency range
        frq = frq[:len(frq)//2] # one side frequency range

        Y = np.fft.fft(y)/n # dft and normalization
        Y = Y[1:n//2]

        yabs = abs(Y)
        xmax = frq[numpy.argmax(yabs)]
        ymax = yabs.max()

        hesitation = xmax
        if hesitation == 0:
            hesitation = -1
    
#    рисование
#    ax3.plot(frq,yabs) # plotting the spectrum
#    ax3.set_xlim([0,30])
#    ax3.set_ylim([Min,0.1])

########################################################
    if flag:
        print('Амплитуда:', amplitude, ' xmax:', xmax, ' ymax:', ymax)
    return [amplitude, xmax, ymax]

Information = []
for df, target in zip(Data,Target):
    Information.append(process_data(df, name=target))

Populating the interactive namespace from numpy and matplotlib
/Users/romanova/Downloads/mini-course-public/Grigory Burmashev - 2021-07-0418.37.42_Бурмашев_стояние.csv
Амплитуда: 0.10399756957898454  xmax: 5.825104780848192  ymax: 0.0003306843557638283
/Users/romanova/Downloads/mini-course-public/Grigory Burmashev - 2021-07-0418.42.06_Бурмашев_стояние.csv
Амплитуда: 0.4699241589391969  xmax: 0.487752005202688  ymax: 0.0006222188973294497
/Users/romanova/Downloads/mini-course-public/Grigory Burmashev - 2021-07-0418.43.28_Бурмашев_стояние.csv
Амплитуда: 3.280368408464594  xmax: 5.510983019161864  ymax: 0.0027026345124821797
/Users/romanova/Downloads/mini-course-public/Grigory Burmashev - 2021-07-0418.50.44_Бурмашев_стояние.csv
Амплитуда: 0.06966254416493811  xmax: 1.4469662482929049  ymax: 0.0005468542938441326
/Users/romanova/Downloads/mini-course-public/Grigory Burmashev - 2021-07-0418.52.08_Бурмашев_стояние.csv
Амплитуда: 0.39952206646925226  xmax: 3.0180460485376477  ymax: 0.00085093

Амплитуда: 0.21060470622832161  xmax: 1.9365199319856414  ymax: 0.00574935571555722
/Users/romanova/Downloads/mini-course-public/Влада Белозерова - 2021-07-0510.13.39 Белозерова стояние.csv
Амплитуда: 0.06277603976687673  xmax: 1.622989874925551  ymax: 0.0004361881295058781
/Users/romanova/Downloads/mini-course-public/Дарья Шагалкина - Шагалкина_стояние_1.csv
Амплитуда: 0.039274341656528766  xmax: 5.209532581165093  ymax: 0.0006012784554642678
/Users/romanova/Downloads/mini-course-public/Дарья Шагалкина - Шагалкина_стояние_2.csv
Амплитуда: 0.13432054278468097  xmax: 4.932683379758589  ymax: 0.0005828623057660282
/Users/romanova/Downloads/mini-course-public/Дарья Шагалкина - Шагалкина_стояние_3.csv
Амплитуда: 0.037397877532800106  xmax: 6.184325744308231  ymax: 0.0007192627130034739
/Users/romanova/Downloads/mini-course-public/Дарья Шагалкина - Шагалкина_стояние_4.csv
Амплитуда: 0.08673568889876138  xmax: 6.050605060506051  ymax: 0.000547911794089189
/Users/romanova/Downloads/mini-cours

Амплитуда: 0.0698529328075389  xmax: 2.2507795996378634  ymax: 0.0004351723706097563
/Users/romanova/Downloads/mini-course-public/Рита Арунова - 2021_07_0418_29_46_Арунова_стояние
Амплитуда: 0.03995955406882035  xmax: 13.098318240620957  ymax: 0.0005235570252666997
/Users/romanova/Downloads/mini-course-public/Рита Арунова - 2021_07_0418_31_23_Арунова_стояние
Амплитуда: 0.05578340464585907  xmax: 8.234519104084322  ymax: 0.0005927502645936857
/Users/romanova/Downloads/mini-course-public/Рита Арунова - 2021_07_0418_32_55_Арунова_стояние
Амплитуда: 0.039785944295872455  xmax: 13.49527665317139  ymax: 0.0006471568583575956
/Users/romanova/Downloads/mini-course-public/Рита Арунова - 2021_07_0418_34_30_Арунова_стояние
Амплитуда: 0.07242025109088635  xmax: 14.696058784235136  ymax: 0.0007470598495896708
/Users/romanova/Downloads/mini-course-public/Роман Гладких - Гладких_стояние_1
Амплитуда: 0.031334528062239686  xmax: 121.21471831790653  ymax: 0.00014421356547992937
/Users/romanova/Downloads

Амплитуда: 0.3061831697697017  xmax: 2.547169811320755  ymax: 0.001771344351665083
/Users/romanova/Downloads/mini-course-public/Алексей - 2021-07-0221.06.41 Голенков Стояние.csv
Амплитуда: 0.847714027734594  xmax: 7.815845824411135  ymax: 0.005020972997843186
/Users/romanova/Downloads/mini-course-public/Алексей Гомбожапов - Гомбожапов_стояние_2.csv
Амплитуда: 0.046136845796718506  xmax: 0.0  ymax: 0.005803469144311819
/Users/romanova/Downloads/mini-course-public/Алексей Гомбожапов - Гомбожапов_стояние_3.csv
Амплитуда: 0.21407771311495893  xmax: 0.0  ymax: 0.05071361653529361
/Users/romanova/Downloads/mini-course-public/Алексей Гомбожапов - Гомбожапов_стояние_4.csv
Амплитуда: 0.14536379055194637  xmax: 0.0  ymax: 0.026650616603474453
/Users/romanova/Downloads/mini-course-public/Алексей Гомбожапов - Гомбожапов_стояние_5.csv
Амплитуда: 0.07942902046584943  xmax: 0.0  ymax: 0.013334351663755559
/Users/romanova/Downloads/mini-course-public/Анастасия Романова - Романова_Стояние_1.csv
Ам

/Users/romanova/Downloads/mini-course-public/Дмитрий Мальков - 2021-07-0720.27.32 Мальков стояние на месте
Амплитуда: 0.6245107309028508  xmax: 5.298651252408478  ymax: 0.0021821794746552793
/Users/romanova/Downloads/mini-course-public/Дмитрий Тихановский - sensor1 Тихановский Стояние на месте.csv
Амплитуда: 0.04743617892684859  xmax: 15.965439519158528  ymax: 0.0008413307703213986
/Users/romanova/Downloads/mini-course-public/Дмитрий Тихановский - sensor2 Тихановский стояние на месте.csv
Амплитуда: 0.0476097684350828  xmax: 7.776049766718507  ymax: 0.0005544333038144445
/Users/romanova/Downloads/mini-course-public/Дмитрий Тихановский - sensor3 Тихановский стояние на месте.csv
Амплитуда: 0.061539604417068405  xmax: 17.714109673444238  ymax: 0.00047369325151253163
/Users/romanova/Downloads/mini-course-public/Дмитрий Тихановский - sensor4 Тихановский стояние на месте.csv
Амплитуда: 0.26017869294057194  xmax: 11.904761904761903  ymax: 0.0009626193670573968
/Users/romanova/Down

/Users/romanova/Downloads/mini-course-public/Мария Крылова - 2_Крылова_стояние_на_месте.csv
Амплитуда: 0.07608888241325673  xmax: 9.785472337222432  ymax: 0.0017320218600311967
/Users/romanova/Downloads/mini-course-public/Мария Крылова - 3_Крылова_стояние_на_месте.csv
Амплитуда: 0.12482923926615475  xmax: 4.502442002442002  ymax: 0.0013595740511719977
/Users/romanova/Downloads/mini-course-public/Мария Крылова - 4_Крылова_стояние_на_месте.csv
Амплитуда: 0.13452735941424665  xmax: 6.352087114337568  ymax: 0.0016024429665656207
/Users/romanova/Downloads/mini-course-public/Мария Крылова - 5_Крылова_стояние_на_месте.csv
Амплитуда: 0.06230320620906593  xmax: 8.04289544235925  ymax: 0.002154540926140105
/Users/romanova/Downloads/mini-course-public/Михаил Никитаев - Никитаев_стояние_1.csv.csv
Амплитуда: 1.1591739528475085  xmax: 5.551069842551474  ymax: 0.004342064794396308
/Users/romanova/Downloads/mini-course-public/Никита Нарджиев - Нарджиев Никита - Нарджиев стояние 1.csv
Амплитуда: 0.0411

Напишем функцию для получения предсказания (здесь используется только амплитуда, но в вашем итоговом решении будет больше признаков)

In [509]:
def get_prediction(info):
    ampl, xmax, ymax = info[0], info[1], info[2]
    if ampl < 1 and ymax < 0.01:
        return 0 #стояние
    if ampl > 5:
        return 2 #бег
    if 1.9 < xmax < 4 and ymax < 0.1:
        return 4 #лестница
    elif ymax > 0.1:
        return 1
    if xmax > 5 and 0.5 > ymax > 0.008:
        return 3 #велосипед
    return -1

In [500]:
len(Data), len(Information), len(Target)

(1196, 1196, 1196)

Некоторые амплитуды посчитаны некорректно, уберём соответствующие треки.

In [501]:
good_indices = [i for i in range(len(Information)) if not ((math.isnan(Information[i][0]) or Information[i][0]==-1) and 
                                                    (math.isnan(Information[i][1]) or Information[i][1]==-1) and
                                                    (math.isnan(Information[i][2]) or Information[i][2]==-1))]


GoodData   = [Data[i] for i in good_indices]
GoodInfo    = [Information[i] for i in good_indices]
GoodTarget = [Target[i] for i in good_indices]

In [502]:
len(Data), len(GoodInfo), len(GoodData), len(GoodTarget)

(1196, 1195, 1195, 1195)

Создадим таблицу, содержащую номер трека, правильный ответ и предсказание.

In [503]:
prediction = [get_prediction(a) for a in GoodInfo]
answer = [get_action(target) for target in GoodTarget]

Res = pd.DataFrame({'track_num' : np.arange(len(GoodData)), 'answer' : answer, 'action' : prediction})
Res.head(10)

,track_num,answer,action
0,0,0,0
1,1,0,0
2,2,0,1
3,3,0,0
4,4,0,0
5,5,1,1
6,6,1,1
7,7,1,1
8,8,1,1
9,9,3,1


Проверим качество нашей модели на тренировочных данных (на данных из открытой папки)

In [504]:
from sklearn.metrics import accuracy_score

accuracy_score(Res['answer'], Res['action'])

0.6343096234309623

В тренировочной папке количество треков разных типов несбалансировано, поэтому для корректной оценки качества нашей модели будем проверять качество модели на таком же распределении треков, как и в тестовой выборке:

* 18% - стояние
* 18 % - ходьба
* 18 % - бег
* 18 % - подъем по лестнице
* 18 % - велосипед
* 10% - другие типы движений

In [505]:
Res['answer'].value_counts(), len(Res)

( 1    299
  0    284
  4    221
  2    216
 -1     62
  3     57
  5     31
  6     13
  7      8
  8      4
 Name: answer, dtype: int64, 1195)

In [506]:
import random

ind_0 = random.sample(Res[Res.answer == 0].index.tolist(), 32)
ind_1 = random.sample(Res[Res.answer == 1].index.tolist(), 32)
ind_2 = random.sample(Res[Res.answer == 2].index.tolist(), 32)
ind_3 = random.sample(Res[Res.answer == 3].index.tolist(), 32)
ind_4 = random.sample(Res[Res.answer == 4].index.tolist(), 32)
ind_5 = random.sample(Res[Res.answer == 5].index.tolist(), 6)
ind_6 = random.sample(Res[Res.answer == 6].index.tolist(), 6)
ind_7 = random.sample(Res[Res.answer == 7].index.tolist(), 6)

ind_balanced = ind_0 + ind_1 + ind_2 + ind_3 + ind_4 + ind_5 + ind_6 + ind_7 
len(ind_balanced)

178

In [507]:
Res_balanced = Res.iloc[ind_balanced]

In [508]:
accuracy_score(Res_balanced['answer'], Res_balanced['action'])

0.601123595505618